# Construindo a nova safra de produção

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_orders = pd.read_csv('/content/drive/MyDrive/projeto-ia-datasets/olist/input/olist_orders_dataset.csv', parse_dates=['order_approved_at'])
df_order_items = pd.read_csv('/content/drive/MyDrive/projeto-ia-datasets/olist/input/olist_order_items_dataset.csv')
df_sellers = pd.read_csv('/content/drive/MyDrive/projeto-ia-datasets/olist/input/olist_sellers_dataset.csv')

In [ ]:
import dateutil

data_ref_safra     = pd.to_datetime('2018-04-01').date()
data_inf_inclusiva = data_ref_safra - dateutil.relativedelta.relativedelta(months=12)

df_producao = (

    # juntando as tabelas e filtrando o período
    df_order_items
    .merge(df_orders, on='order_id', how='inner')
    .query('order_status == "delivered"')
    .query(f'order_approved_at >= "{data_inf_inclusiva}" & order_approved_at < "{data_ref_safra}"')
    .merge(df_sellers, on='seller_id', how='left')

    # construindo as features
    .groupby('seller_id')
    .agg(uf                 = ('seller_state', 'first'),
        tot_orders_12m     = ('order_id', 'nunique'),
        tot_items_12m      = ('product_id', 'count'),
        tot_items_dist_12m = ('product_id', 'nunique'),
        receita_12m        = ('price', 'sum'),
        data_ult_vnd       = ('order_approved_at', 'max'))
    .reset_index()
    .assign(data_ref_safra = pd.to_datetime(f'{data_ref_safra}'))
    .assign(recencia = lambda df: (df['data_ref_safra'] - df['data_ult_vnd']).dt.days)
    .filter(['data_ref_safra', 'seller_id', 'uf', 'tot_orders_12m', 'tot_items_12m', 'tot_items_dist_12m', 'receita_12m', 'recencia'])
)

In [ ]:
df_producao

,data_ref_safra,seller_id,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,recencia
0,2018-04-01,0015a82c2db000af6aaaf3ae2ecb0532,SP,3,3,1,2685.00,164
1,2018-04-01,001cca7ae9ae17fb1caed9dfb1094831,ES,171,195,10,20321.53,0
2,2018-04-01,002100f778ceb8431b7a1020ff7ab48f,SP,48,52,23,1106.80,4
3,2018-04-01,003554e2dce176b5555353e4f3555ac8,GO,1,1,1,120.00,106
4,2018-04-01,004c9cd9d87a3c30c522c48c4fc07416,SP,126,131,65,14685.67,5
...,...,...,...,...,...,...,...,...
1936,2018-04-01,ffad1e7127fb622cb64a900751590acd,SP,1,2,1,35.60,18
1937,2018-04-01,ffc470761de7d0232558ba5e786e57b7,SP,6,7,6,555.57,25
1938,2018-04-01,ffdd9f82b9a447f6f8d4b91554cc7dd3,PR,10,11,8,1350.30,38
1939,2018-04-01,ffeee66ac5d5a62fe688b9d26f83f534,SP,13,13,3,1709.87,31


Perceba que não temos o target, pois essa base é de produção (evento de interesse ainda não ocorreu).

In [ ]:
key_vars = ['data_ref_safra', 'seller_id']
num_vars = ['tot_orders_12m', 'tot_items_12m', 'tot_items_dist_12m', 'receita_12m', 'recencia']
cat_vars = ['uf']

features = cat_vars + num_vars

X_producao = df_producao[features]

In [ ]:
X_producao

,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,recencia
0,SP,3,3,1,2685.00,164
1,ES,171,195,10,20321.53,0
2,SP,48,52,23,1106.80,4
3,GO,1,1,1,120.00,106
4,SP,126,131,65,14685.67,5
...,...,...,...,...,...,...
1936,SP,1,2,1,35.60,18
1937,SP,6,7,6,555.57,25
1938,PR,10,11,8,1350.30,38
1939,SP,13,13,3,1709.87,31


# Carregando o modelo

In [ ]:
!pip install feature_engine==1.0.2

In [ ]:
!pip install catboost==0.25.1

In [ ]:
import joblib

model = joblib.load('/content/drive/MyDrive/projeto-ia-datasets/olist/output/catboost_model_20210414.pkl')

In [ ]:
model

Pipeline(memory=None,
         steps=[('numeric_imputer',
                 ArbitraryNumberImputer(arbitrary_number=-999,
                                        imputer_dict=None,
                                        variables=['tot_orders_12m',
                                                   'tot_items_12m',
                                                   'tot_items_dist_12m',
                                                   'receita_12m',
                                                   'recencia'])),
                ('categoric_imputer',
                 CategoricalImputer(fill_value='missing',
                                    imputation_method='missing',
                                    return_object=False, variables=['uf'])),
                ('one_hot_encoder',
                 OneHotEncoder(drop_last=False, top_categories=None,
                               variables=['uf'])),
                ('algoritmo',
                 <catboost.core.CatBoostClassifier obj

# Aplicando o Modelo nos Novos Dados

In [ ]:
y_proba_producao = model.predict_proba(X_producao)[:,1]

In [ ]:
df_producao.assign(score = y_proba_producao)

,data_ref_safra,seller_id,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,recencia,score
0,2018-04-01,0015a82c2db000af6aaaf3ae2ecb0532,SP,3,3,1,2685.00,164,0.855507
1,2018-04-01,001cca7ae9ae17fb1caed9dfb1094831,ES,171,195,10,20321.53,0,0.004043
2,2018-04-01,002100f778ceb8431b7a1020ff7ab48f,SP,48,52,23,1106.80,4,0.061906
3,2018-04-01,003554e2dce176b5555353e4f3555ac8,GO,1,1,1,120.00,106,0.798596
4,2018-04-01,004c9cd9d87a3c30c522c48c4fc07416,SP,126,131,65,14685.67,5,0.002224
...,...,...,...,...,...,...,...,...,...
1936,2018-04-01,ffad1e7127fb622cb64a900751590acd,SP,1,2,1,35.60,18,0.324999
1937,2018-04-01,ffc470761de7d0232558ba5e786e57b7,SP,6,7,6,555.57,25,0.109447
1938,2018-04-01,ffdd9f82b9a447f6f8d4b91554cc7dd3,PR,10,11,8,1350.30,38,0.268410
1939,2018-04-01,ffeee66ac5d5a62fe688b9d26f83f534,SP,13,13,3,1709.87,31,0.234630


# Monitoramento de Modelos - Introdução

- Como saber se o modelo irá performar bem ou não nos novos dados? Podemos pensar em duas formas:

    * Esperar passar os 6 meses a partir do dia 2018-04-01 e calcular a métrica de interesse
    * Verificar se a distribuição da predição/score mudou na base nova (produção) em relação a base de treinamento (referência). Quando há uma mudança de distribuição da variável, dizemos tecnicamente que ocorreu um **drift**.

Na primeira opção temos um monitoramento reativo, pois esperamos o evento acontecer para poder calcular a performance do modelo. Geralmente essa opção não é interessante, pois em termos de negócios o ideal é atuarmos antes do modelo perder a performance, ou seja, de forma preventiva. A segunda opção é uma forma de se fazer isso.

Para implementarmos a segunda opção, iremos utilizar uma biblioteca do python chamada [evidently](https://github.com/evidentlyai/evidently).

In [ ]:
!pip install evidently==0.1.14.dev0

In [ ]:
from evidently.dashboard import Dashboard
from evidently.tabs import DriftTab

In [ ]:
df_referencia = pd.read_csv('/content/drive/MyDrive/projeto-ia-datasets/olist/output/propensao_revenda_abt.csv')
X_referencia = df_referencia[features]
y_prob_referencia = model.predict_proba(X_referencia)[:,1]
X_referencia['score'] = y_prob_referencia
X_referencia

,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,recencia,score
0,SP,3,3,1,2685.00,74,0.652528
1,ES,171,207,9,21275.23,2,0.004396
2,SP,38,42,15,781.80,2,0.040189
3,GO,1,1,1,120.00,16,0.618263
4,SP,130,141,75,16228.88,8,0.002171
...,...,...,...,...,...,...,...
5364,MG,4,4,3,124.60,12,0.204421
5365,SP,5,5,5,385.59,0,0.062160
5366,PR,11,12,8,1450.20,7,0.081413
5367,SP,13,13,3,1709.87,0,0.061171


In [ ]:
y_prob_producao = model.predict_proba(X_producao)[:,1]
X_producao['score'] = y_prob_producao
X_producao

,uf,tot_orders_12m,tot_items_12m,tot_items_dist_12m,receita_12m,recencia,score
0,SP,3,3,1,2685.00,164,0.855507
1,ES,171,195,10,20321.53,0,0.004043
2,SP,48,52,23,1106.80,4,0.061906
3,GO,1,1,1,120.00,106,0.798596
4,SP,126,131,65,14685.67,5,0.002224
...,...,...,...,...,...,...,...
1936,SP,1,2,1,35.60,18,0.324999
1937,SP,6,7,6,555.57,25,0.109447
1938,PR,10,11,8,1350.30,38,0.268410
1939,SP,13,13,3,1709.87,31,0.234630


O `evidently` só funciona diretamente com variáveis numéricas. Para também aplicar em variáveis categóricas, temos que transformá-las para número e evidenciar isso para o evidently através de um dicionário chamado `column_mapping`

In [ ]:
from feature_engine.encoding import OrdinalEncoder

ordinal_encoder = OrdinalEncoder(encoding_method='arbitrary', variables=cat_vars)
X_referencia = ordinal_encoder.fit_transform(X_referencia)
X_producao = ordinal_encoder.transform(X_producao)

In [ ]:
ordinal_encoder.encoder_dict_

{'uf': {'AM': 13,
  'BA': 16,
  'CE': 15,
  'DF': 10,
  'ES': 1,
  'GO': 2,
  'MA': 20,
  'MG': 4,
  'MS': 18,
  'MT': 12,
  'PA': 19,
  'PB': 9,
  'PE': 11,
  'PI': 21,
  'PR': 3,
  'RJ': 7,
  'RN': 5,
  'RO': 14,
  'RS': 8,
  'SC': 6,
  'SE': 17,
  'SP': 0}}

In [ ]:
# indicando quais são as variáveis numéricas e quais são variáveis categóricas
column_mapping = {}
column_mapping['numerical_features'] = num_vars + ['score']
column_mapping['categorical_features'] = cat_vars

# Irá verificar se a distribuição do score e das variáveis numéricas mudaram na base atual (produção)
# em comparação com a base de treinamento (referência)
# Para calcular o drift entre as distribuições, o evidently utiliza o teste two-sample Kolmogorov-Smirnov e para 
# as variáveis categóricas utiliza o teste chi-square.
drift_report = Dashboard(reference_data=X_referencia, production_data=X_producao, column_mapping=column_mapping, tabs=[DriftTab])

In [ ]:
drift_report.save('/content/drive/MyDrive/projeto-ia-datasets/olist/output/drift_report.html')

Abrindo o relatório `drift_report.html`, iremos olhar primeiro se há algum drift no score predito pelo modelo. Se não há drift no score, esse é um forte indicativo de que o modelo irá manter a estabilidade e não haverá uma perda de performance na nova safra (produção). Agora se há drift no score, é o caso então de olharmos quais features apresentam os maiores drift, pois essas features provavelmente serão as maiores responsáveis pela perda de estabilidade e performance do modelo. Nesse caso, uma boa ação a ser tomada é o retreinamento do modelo com novos dados, se disponível.

No nosso caso, não identificamos drift na predição do modelo. Logo, podemos esperar que o nosso modelo não irá perder performance e manterá a sua estabilidade.